# Predict weather
For renewable generation, predict:
- Irradiance (GHI)
- Temperature
- Wind speed

In [ ]:
#!pip install darts
#!pip install pandas
#!pip install numpy
#!pip install plotly

#!pip install pyyaml==5.4.1 # Google colab

#GPU: https://unit8co.github.io/darts/userguide/gpu_and_tpu_usage.html

#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
#!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchtext==0.10.0 -f https://download.pytorch.org/whl/cu111/torch_stable.html
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
#!python -m pip uninstall matplotlib
#!pip install matplotlib==3.1.3

In [ ]:
import darts as dts
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
from darts.utils.timeseries_generation import datetime_attribute_timeseries as dt_attr
#from darts import concatenate

In [ ]:
# df = pd.read_csv('dataset1_tropical.csv', sep=";")
#df['datetime'] = pd.to_datetime(df['datetime'])

# load data
df2017 = pd.read_csv('data/361685_47.41_0.78_2017.csv', skiprows=2)
df2018 = pd.read_csv('data/361685_47.41_0.78_2018.csv', skiprows=2)
df2019 = pd.read_csv('data/361685_47.41_0.78_2019.csv', skiprows=2)

#concatenate in a whole dataset
df = pd.concat([df2017, df2018, df2019])

#genenarate a datetime index column
df['datetime'] = pd.to_datetime(df['Year'].astype(str) +'-' 
                                + df['Month'].astype(str) 
                                + '-' + df['Day'].astype(str) 
                                + ' ' + df['Hour'].astype(str) 
                                + ':' + df['Minute'].astype(str) + ':00'
                                )

#set datetiem as index
df = df.set_index('datetime')

#drop useless columns
df = df.drop(['Year', 'Month', 'Day', 'Hour', 'Minute'], axis=1)

# accelerate processing by reducing information
#df = df.astype(np.float32)

In [ ]:
df.info()
# df.isnull().sum()  # missing value

In [ ]:
frequency = pd.Timedelta('30 min')
df_resampled = df.resample(frequency).mean()
df_resampled.info()

In [ ]:
fig = px.imshow(df_resampled.corr())
fig.show()

In [ ]:
fig = px.imshow(df.between_time('9:00','17:00').corr())
fig.show()

In [ ]:
fig = px.line(df_resampled.loc['2017-1-1':'2017-12-31'])
fig.show()

In [ ]:
N_DAY = 96
N_PRED = int(N_DAY * 1.5)

In [ ]:
# extract explainable variables
series_ghi = dts.timeseries.TimeSeries.from_series(df['GHI'])
#series_pressure = dts.timeseries.TimeSeries.from_series(df['Pressure'])
#series_water = dts.timeseries.TimeSeries.from_series(df['Precipitable Water'])
#series_wddir= dts.timeseries.TimeSeries.from_series(df['Wind Direction'])
series_wdspeed = dts.timeseries.TimeSeries.from_series(df['Wind Speed'])

In [ ]:
# normalize to use deep learning
from darts.dataprocessing.transformers import Scaler
scaler_ghi, scaler_wdspeed = Scaler(), Scaler()
# series_scaled
series_s_ghi = scaler_ghi.fit_transform(series_ghi)
series_s_wdspeed = scaler_wdspeed.fit_transform(series_wdspeed)

# to inverse scaled with
# scaler_ghi.inverse_transform(series_s_ghi).plot()

In [ ]:
series_s_ghi.plot()

In [ ]:
#fig = px.line(series_s_ghi.pd_dataframe())
#fig.show()

In [ ]:
# set covariate

# covariate for renewable generation
renew_covs = dts.concatenate(
    [
        # df.index.month,
        dt_attr(df.index, "month", dtype=np.float32) / 12,  # month for seasons
        dt_attr(df.index, "hour", dtype=np.float32) / 23,  # hours for sun light
    ],
    axis="component",
)

# covariation for load consumption
load_covs = dts.concatenate(
    [
        dt_attr(df.index, "month", dtype=np.float32) / 12,  # month for seasons
        dt_attr(df.index, "hour", dtype=np.float32) / 23,  # hours for sun light
        dt_attr(df.index, "dayofweek", dtype=np.float32) / \
        6  # for human consumption
    ],
    axis="component",
)
load_covs = load_covs.add_holidays("FR")  # for human consumption

In [ ]:
test = dt_attr(df.index, "month", dtype=np.float32) / 12
test[1:96*366].plot()

In [ ]:
test = dt_attr(df.index, "hour", dtype=np.float32) / 23
test[1:96*4].plot()

In [ ]:
# concatenate a multivariate
from darts import concatenate
multivar_ghi = concatenate([series_s_ghi, series_s_wdspeed], axis=1)

In [ ]:
# acf
from darts.utils.statistics import plot_acf, check_seasonality, plot_pacf
plot_acf(series_ghi, m=96, alpha=0.05, max_lag=96*3)
plot_pacf(series_ghi, m=96, alpha=0.05, max_lag=96*3)

In [ ]:
train, test = series_s_ghi.split_before(2./3.)
train.plot(label="train")
test.plot(label="test")

In [ ]:
## ARIMA
#https://towardsdatascience.com/creating-an-arima-model-for-time-series-forecasting-ff3b619b848d

from darts.models.forecasting.arima import ARIMA

model = ARIMA(p=N_DAY)

# fit the model
model.fit(series=train[1:N_DAY*2])#,
          # past_covariates=renew_covs)#,
          #future_covariates=renew_covs)

In [ ]:
pred = model.predict(n=N_PRED)#,
                     #series=test[1:N_DAY*15])#,  # this is only required for GFMs
                     #past_covariates=past_covariates,
                     #future_covariates=renew_covs)#,
                     #num_samples = 1)

In [ ]:
train[:N_DAY*15].plot(label="train")
pred.plot(label="test")

In [ ]:
from darts.metrics import coefficient_of_variation
OFFSET = N_DAY*2 + N_PRED
coefficient_of_variation(train[N_DAY*2:OFFSET],pred)

In [ ]:
## TFT

In [ ]:
from darts.models.forecasting.tft_model import TFTModel
#GPU: https://unit8co.github.io/darts/userguide/gpu_and_tpu_usage.html

# create one of Darts' forecasting model
model = TFTModel(input_chunk_length=N_DAY * 7,
                 output_chunk_length=N_DAY,
                  force_reset=True,
                 #pl_trainer_kwargs={
                 #    "accelerator": "gpu",
                 #    "devices": [0]
                 #}
                 )

# fit the model
model.fit(series=train[1:N_DAY*15],
          # past_covariates=renew_covs)#,
          future_covariates=renew_covs)


In [ ]:
# make a prediction with the same covariate types
pred = model.predict(n=N_DAY,
                     series=test,  # this is only required for GFMs
                     #past_covariates=past_covariates,
                     future_covariates=renew_covs)